# Vessel Manoeuvring Models
Many simulation model for ship manoeuvring have been developed in the field of ship hydrodynamics such as: the Abkowitz model {cite:p}`abkowitz_ship_1964` or the Norrbin model {cite:p}`norrbin_study_1960`.
This chapter will develop a general simulation model for ship manoeuvring, that can be further specified to become either the Abkowitz or Norbin model. Expressing the models on a general form is important in this research where many different models will be tested and compared.

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmm_names = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

vmms = {}
for vmm_name in vmm_names:
    vmms[vmm_name] = catalog.load(vmm_name)


The vessel manoeuvring models can be expressed in a very general way ( {cite:p}`fossen_handbook_2021`):

In [ ]:
eq_6DOF

Where $\eta$ describes the position:

In [ ]:
eq_eta

and $\nu$ is the velocities:

In [ ]:
eq_nu

The accelerations are denoted using the dotted notation: $\dot{\nu}$.

* $M$ is inertia matrix
* $C(\nu)$ is corriolis/centrepetal matrix as function of the velocities \nu.
* $D(\nu)$ is damping matrix as a function of vecocities \nu.
* $g(\eta)$ is a vector of generalized gravitational an buoyance forces.
* $g_0$ is static restoring forces due to ballast systems.
* $\tau$ is vector of control inputs (from rudders/propellers etc.)
* $\tau_{wind}$ is vector of wind forces
* $\tau_{wave}$ is vector of wave forces

The velocities can also include the ocean current by expressing the relative velocity $v_r$ as:

In [ ]:
eq_nu_r

If the current is assumed to be irrotational, the angular velocities for the current is zero:

In [ ]:
eq_nu_c

In [ ]:
eq_nu_r_expanded

If the current is also assumed to be constant, this mean that the time derivative of $\nu$ and $\nu_r$ are the same, also giving the same accelerations: 

In [ ]:
eq_nu_steady

When current is present the inertia as well as the corriolis matrix must be split into an added mass part (A) and a rigid body part (RB):

In [ ]:
eq_M

In [ ]:
eq_C

So that the model equation can be written as:

In [ ]:
eq_6DOF_expanded

This equation can be simplified for manoeuvring models by firstly only keeping surge, sway and yaw degrees of freedome. This means that both $g_0$ and $g(\eta)$ dissapears as there are no static forces for surge, sway and yaw. For the classic manoeuvring problem also forces from wind and waves are neglected, removing $\tau_{wave}$ and $\tau_{wind}$. In this paper, there are no ocean current during the studied model tests, so that the relative velocity $v_r$ can be replaced with $v_r$.

In [ ]:
eq_3DOF = eq_6DOF_expanded.subs([

    (g_0,0),
    (g_function, 0),
    (tau_wave,0),
    (tau_wind,0),
    (nu_r,nu)

])
eq_3DOF

And these matrices can be expressed in 3 degrees of freeddome ( {cite:p}`fossen_handbook_2021`):

In [ ]:
eq_C_A

In [ ]:
eq_C_RB

In [ ]:
eq_M_A

In [ ]:
eq_M_RB

The damping and control inputs from rudders and propellers are replaced with the functions $X_D(u,v,r,\delta,thrust)$, $Y_D(u,v,r,\delta,thrust)$, $N_D(u,v,r,\delta,thrust)$. Note that the measured thrust from the model tests is used as input to the models, which means that the propeller is not part of the models in this paper. Main focus is thereby on the modelling of rudder and hull forces and moments.

In [ ]:
eq_D_function

In [ ]:
CD_ = (eq_C_A.rhs + eq_C_RB.rhs).doit()

The system equation can now be written as:

In [ ]:
eq_system

This equation can be rewritten to get the acceleration on the left hand side:

In [ ]:
eq_acceleration_matrix_clean

where $S$ is a helper variable:

In [ ]:
eq_S

A state space model for manoeuvring can now be defined with six states:

In [ ]:
eq_x

An transition function $f$ defines how the states changes with time:

In [ ]:
eq_state_space

Using geometrical relations for how $x_0$, $y_0$ and $\Psi$ depend on $u$, $v$, and $r$ and the time derivatives that was derived above: $\dot{u}$, $\dot{v}$, $\dot{r}$, the transition function can be written:

In [ ]:
eq_f

The manoeuvring simulation can now be conducted by numerical integration of the above equation. The main difference between various vessel manoeuvring models such as the Abkowitz model {cite:p}`abkowitz_ship_1964` or the Norrbin model {cite:p}`norrbin_study_1960` lies in how the hydrodynamic functions $X_D(u,v,r,\delta,thrust)$, $Y_D(u,v,r,\delta,thrust)$, $N_D(u,v,r,\delta,thrust)$ are defined. These functions cane be found in [Appendix](appendix_vmms.md).

Note that a coefficient $X_{thrust}$ has been added to the Abkowitz X equation to allow for propeller thrust as an input to the model. 

In [ ]:
from wPCC_pipeline.jupyter_book import Appendix
appendix = Appendix(file_path='appendix_vmms.md', title='Vessel Manoeuvring Models')

for vmm_name, vmm in vmms.items():
    
    appendix.add_header(vmm_name, level=3)
    eqs = {'X_D':vmm.X_qs_eq,
           'Y_D':vmm.Y_qs_eq,
           'N_D':vmm.N_qs_eq,
          }
    
    for name, eq in eqs.items():
        label = f"eq_{name}_{vmm_name}"
        appendix.add_equation_multiline(eq=eq, label=label)